# Funciones que devuelven funciones

----------------------------------

En anteriores lecciones hemos visto que las funciones también son un tipo de Python:

In [3]:
def foo(n):
    return n

type(foo)

function

##### Los Tipos tienen instancias

Los *tipos* tienen un conjunto de *instancias* que puede ser finito o infinito (más o menos). 

Un ejemplo de tipo con infinitas instancias, sería `str`:  todas las cadenas que se te puedan ocurrir.

Las *funciones* son un *tipo* con infinitas instancias también: todas las funciones que vienen en Python, y todas las que se te ocurran a ti.

##### Los tipos tienen operaciones comunes

Todos los *tipos* tienen algunas operaciones comunes que se pueden hacer con ellos.

Por ejemplo, todos los `int` se pueden sumar.

En el caso de las funciones, tenemos dos operaciones comunes:

* llamar
* componer



##### Las instancias de un tipo son un valor

Un valor es algo que no puede ser simplificado más por parte del lenguaje. Algunso ejemplos son:

* un entero o un decimal
* una cadena
* una lista
* en general, toda instancia de un tipo es un valor

> Si una función devuelve un valor y una función es en sí un valor,
> ¿una función podría devolver otra función? **SI**



#### Funciones que devuelven funciones

Supongamos que necesitas una función que le suma 1 a su parámetro. Podrías definirla dela siguiente manera:



In [ ]:
def add1(n):
    return n + 1

Pasado un tiempo, resulta que necesitas la función que suma 42 a su parámetro:

In [ ]:
def add42(n):
    return n + 42

Ahora resulta que necesitas la función que suma 39 a su parámetro. Esto empieza ya a ponerse pesado y los dioses del Principio DRY empiezan a cabrearse.

Deberíamos de poder *automatizar* la creación de esas funciones tan parecidas entre sí.

Ese *automatizar* significa: *crear una función, que recibe un parámetro (el número a sumar) y devuelve la función de suma*.

Vamos a crear la función `add(m)` que recibe un parámetro `m` y devuelve *una función que acepta un parámetro y le suma `m`*.



In [4]:
def add(m):
    """
    Recibe un número (m) y devuelve la función que acepta un parámetro y le suma m.
    """
    def _add(n):
        """
        Función interna que sirve de patrón.
        LO que vamos a devolver es una versión 
        especializada de ella
        """
        return m + n
    return _add

¿Qué devuelven las siguientes expresiones?

1. `add`
2. `add(1)`
3. `add(1)(4)`
4. `add(1)(-1)`

¿cual es la diferencia entre la función `add` de arriba y la siguiente:

In [ ]:
def addd(m,n):
    return m + n

#### Redefinir `add1` y `add42`

Ahora que tenemos la función que crear sumadores, podemos redefinir `add1` y `add42` en base a `add`:

In [5]:
add1 = add(1)
add42 = add(42)


# comprueba que funcionan correctamente
print(add1(8))
print(add42(42))

9
84


#### 🛎️ 🧢  ¿Qué pasa con `m`?

`m` es el parámetro de la función `add` y la función interna `_add` lo *ve* porque está en el ámbito de la función `add`.

1. Cuando se devuelve la función interna `_add` ésta *se lleva consigo una **copia** de m*. 
2. Cada versión de `_add` devuelta se lleva su propia copia, distinta de las demás.

Cuando hacemos 

```Python
add22 = add(22)
add7 = add(7)
```

Tanto `add22` como `add7` tiene su propia versión de `m`, con valores de `22` y `7` respectivamente.

> `_add` se lleva una copia del ámbito en el que es creada




#### 🛎️ 🧢  Clausura Léxica

Cuando una función se lleva una copia del ámbito en el que ha sido creada, se dice que es una *clausura léxica*.  

Todas las funciones de Python tienen ese superpoder y es común en la mayoría de los lenguajes modernos, como Swift, Kotlin, Javascript, Scala y otros muchos. Se trta de algo que resulta muy cómodo y potente.

Lo usareis con frecuencia en Mobile y Web.


#### 🛎️ 🧢  Memoria entre llamada y llamada

Una de las características de una función, es que su ámbito se crea cuando se le llama y se destruye cuando devuelve.

Por ejemplo, lo siguiente función siempre devuelve `1`, no importa cuantas veces se le llame.


In [ ]:
def inc():
    counter = 0
    counter = counter + 1
    return counter

La razón es que `counter` siempre es recreado con valor cero cada vez que se llama a `inc()`.

> La captura del entorno léxico, puede cambiar esto


Vamos a crear una nueva función que crea funciones similares a `inc`.

In [10]:
def inc_maker():
    """
    Devuelve una función que va incrementando un contador
    """
    counter = 0 # creamos un contador a cero
    def _inc():
        nonlocal counter        # 1
        counter = counter + 1   # 2
        return counter
    return _inc

Aquí aparece algo nuevo (marcado con un 1) que tenemos que explicar.

Cuando Python se encuentra con algo como:

```Python
name = "Lucas"
```

No está muy claro lo que debe de hacer:

1. ¿Creo una nueva variable en el ámbito actual y le meto el valor "Lucas"?
2. ¿Cambio el valor de una variable pre-existente (en el ámbito actual o alguno exterior) llamada `name`?

La confusión viene del hecho de que Python *usa la misma sintaxis para las dos cosas*. La mayoría de los lenguajes con los
que te vas a encontrar no padecen de esto, al tener sitaxis diferentes para cada caso.

Cuando se encuentra con eso, lo que Python hace por defecto, sin pensar demasiado, es **crear una nueva variable** llamada `name`. Si había otra variable con ese nombre en un ámbito externo, le hará sombra y no la modificará.

Por ejemplo, en este caso:

```Python
name = 'Guido' # creo una nueva variable
def foo()->None:
    name = 'Manolo' # creo otra variable local a éste ámbito
    print(name)
```

Tras llamar a `foo`, la variable en el ámbito global sigue inalterada.

##### `global`

Ahora bien, supongamos que el código de `foo`fuese el siguiente:

```Python
name = 'Guido' # creo una nueva variable
def foo()->None:
    global name   # aviso que no quiero crear una nueva variable
    name = 'Manolo' # sino que quiero alterar una global
    print(name)
```

Al llamar a `foo`, se va a *cambiar la variable global*. 
**Nunca hagas esto**.


##### `nonlocal`

Si la variable pre-existente que queremos cambiar no es *global*, sino que está en un ámbito superior (sin llegar al global), hay otra palabra clave para indicar nuestro deseo de modificarla: `nonlocal`.

Esto es lo que ocurre en esta función:

```Python
def inc_maker():
    """
    Devuelve una función que va incrementando un contador
    """
    counter = 0 # creamos una variable en este ámbito y le damos el valor 0
    def _inc():
        nonlocal counter        # Aviso que no quiero crea una nueva en este ámbito
        counter = counter + 1   # sino cambiar la superior
        return counter
    return _inc
```



#### Qué ocurre cuando `inc_maker` devuelve un `_inc`

**Cada `inc` se lleva su propia copia de `counter`**.

Por eso se dice que *las clausuras se llevan una copia de su entorno léxico*.

![](clausuras_entorno_lexico.png)



In [13]:
c1 = inc_maker() # Un _inc con su propio counter
c2 = inc_maker() # Otro _inc con un counter propio, distinto del anterior

c1()    # su counter vale 1
c1()    # su counter ahora vale 2
c2()    # su counter vale 1

1

Mediante esta triquiñuela, las funciones `_inc` *tienen memoria* y su estado se preserva de una llamada a otra. Podemso saber cunatas veces han sido llamadas, por el valor de su varaible capturada.

Esto es mucho más útil de lo que parece ahora mismo y lo usarás mucho en el futuro en lenguajes como Swift o Javascript.